In [1]:
import pymeshlab
import numpy as np

In [2]:
ms = pymeshlab.MeshSet()

ms.load_new_mesh("model_normalized.obj")

m = ms.current_mesh()

v_matrix = m.vertex_matrix()
f_matrix = m.face_matrix()

By default faces are defined and stored in the counter-clockwise order, so should not have any issues calculating the moments

In [3]:
def C_ijk(C,i,j,k):
    return (C[0]**i)*(C[1]**j)*(C[2]**k)*np.math.factorial(i+j+k)/(np.math.factorial(i)*np.math.factorial(j)*np.math.factorial(k))
        

def D_ijk(D_tensor,B,C,i,j,k):
    if (i<0) or (j<0) or (k<0):
        # D_ijk=0
        pass
    elif (0==i) and (0==j) and (0==k):
        # D_ijk = 1
        D_tensor[i,j,k] = 1
    else:
        D_tensor[i,j,k] = B[0]*D_tensor[i-1,j,k]+B[1]*D_tensor[i,j-1,k]+B[2]*D_tensor[i,j,k-1]+C_ijk(C,i,j,k)

def S_ijk(S_tensor,D_tensor,A,B,C,i,j,k):
    if (i<0) or (j<0) or (k<0):
        # S_ijk = 0
        pass
    elif (0==i) and (0==j) and (0==k):
        # S_ijk = 1
        S_tensor[i,j,k] = 1
    else:
        S_tensor[i,j,k] = A[0]*S_tensor[i-1,j,k]+A[1]*S_tensor[i,j-1,k]+A[2]*S_tensor[i,j,k-1]+D_tensor[i,j,k]
    
        
def vol(A,B,C):
    return 1./6*np.linalg.det([A,B,C])


def M_ijk(f_matrix, v_matrix, max_m):
    M_tensor = np.zeros([max_m,max_m,max_m])
    
    
    for face_num in range(len(f_matrix)):
        
        
        # Get the three coordinates A,B,C
        # Any mesh library should return these in CCW order
        [A,B,C] = v_matrix[f_matrix[face_num]]
        
        # Calculate the face moments tensor
        # Each face moments tensor requires calculating S_ijk, D_ijk, and C_ijk Tensor Calculations.

        # C_ijk tensor is not reused, calculate on the fly
        
        # D_ijk tensor is reused, calculate and store
        D_tensor = np.zeros([max_m,max_m,max_m])
        for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        D_ijk(D_tensor,B,C,i,j,k)
                        
        # S_ijk tensor is reused, calculate and store
        S_tensor = np.zeros([max_m,max_m,max_m])
        for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        S_ijk(S_tensor,D_tensor,A,B,C,i,j,k)
                        
        # Now calculate the moments on current face
        f_M_tensor = np.zeros([max_m,max_m,max_m])
        for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        f_M_tensor[i,j,k] = (np.math.factorial(i)*np.math.factorial(j)*np.math.factorial(k))/np.math.factorial(i+j+k+3)*np.linalg.det([A,B,C])*S_tensor[i,j,k]
        
        # Add contribution to the total moments
        M_tensor = M_tensor+f_M_tensor
                
    return M_tensor



This is the final loss function that gives the scalar.

In [4]:
def mesh_moment_loss(f_matrix_1, v_matrix_1, f_matrix_2, v_matrix_2, max_m = 5):
    return np.linalg.norm(M_ijk(f_matrix_1, v_matrix_1,max_m)-M_ijk(f_matrix_2, v_matrix_2,max_m))

In [5]:
# Make it invariant to translation and scaling

def mesh_moment_invariant(f_matrix_1, v_matrix_1, max_m = 5):
    
    # Obtain the scale and centroids of the object
    Tensor_1 = M_ijk(f_matrix_1, v_matrix_1,2)
    print(Tensor_1)
    
    v_matrix_1 = np.array(v_matrix_1)
    v_matrix_1 = v_matrix_1 / np.cbrt(Tensor_1[0,0,0])
    
    # Obtain the scale and centroids of the object again for better numerical precision
    # Doing this recursively keeps improving precision of the scale and centroids
    Tensor_1 = M_ijk(f_matrix_1, v_matrix_1,2)
    print(Tensor_1)
    
    v_matrix_1 = np.array(v_matrix_1)
    v_matrix_1 = v_matrix_1 / np.cbrt(Tensor_1[0,0,0])
    
#     v_matrix_1[:,0] = v_matrix_1[:,0] - Tensor_1[1,0,0]/Tensor_1[0,0,0] 
#     v_matrix_1[:,1] = v_matrix_1[:,1] - Tensor_1[0,1,0]/Tensor_1[0,0,0] 
#     v_matrix_1[:,2] = v_matrix_1[:,2] - Tensor_1[0,0,1]/Tensor_1[0,0,0] 
    

    M_1 = M_ijk(f_matrix_1, v_matrix_1,max_m)
    
#     for i in range(max_m):
#             for j in range(max_m):
#                 for k in range(max_m):
#                     if (i+j+k)<=max_m:
#                         M_1[i,j,k] = M_1[i,j,k] / np.power(Tensor_1[0,0,0],(i+j+k+3)/3)
    
    return M_1

Note that this loss function is directly comparing the moments.

Matching the moment series will make two shapes match each other.

However, if we want to make the loss function invariant to scale, translation, and rotation, (or any symmetries) then the invariants of the loss function can be calcuated from the moments tensor we have just calculated.
Matching the invariants will make two shapes "match" each other without being affected by scale, translation, and rotation.

This is easy to implement, will do this next.

In [6]:
mesh_moment_invariant(f_matrix,v_matrix,max_m=3)

[[[-1.98788045e-20  2.92615142e-19]
  [ 7.66589633e-21  9.39987369e-21]]

 [[ 2.54016826e-21 -5.20477551e-23]
  [-7.89763615e-23  0.00000000e+00]]]
[[[-1.58700684e+02  6.42263040e+07]
  [-5.75091200e+06  4.02307573e+13]]

 [[-1.16475200e+06 -1.31755672e+12]
  [ 8.73254093e+10  0.00000000e+00]]]


array([[[-1.35017395e-01, -8.79705000e+04, -3.34200832e+08],
        [ 8.69662500e+03,  1.04837120e+09,  5.87311008e+14],
        [ 1.69082880e+07, -3.88895867e+13,  0.00000000e+00]],

       [[-1.87500000e+02, -1.24810240e+08, -2.66356423e+13],
        [ 1.63225600e+07,  6.98885131e+12,  0.00000000e+00],
        [-4.38355100e+11,  0.00000000e+00,  0.00000000e+00]],

       [[-4.37125120e+07, -2.37038574e+12,  0.00000000e+00],
        [ 5.66671442e+11,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]]])

In [5]:
M_ijk(f_matrix, v_matrix,4)

SOMETHING IS WRONG WITH THE MESH
THE VOLUME SHOULD NOT BE NEGATIVE

IS PYMESHLAB LIBRARY FOLLOWING THE CCW CONVENTION????

SyntaxError: invalid syntax (3798961729.py, line 3)